In [ ]:
import ee
# authentication
service_account ='id-80b-capstone@a-capstone.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, "a-capstone-fec01f15055b.json")
ee.Initialize(credentials)

In [ ]:
import folium
import geemap
import os
import IPython.display as disp
from IPython.display import Image
import glob

In [ ]:
# !pip install rasterio
import rasterio #for reading images
from skimage.transform import resize

In [ ]:
from sklearn import cluster
from sklearn.metrics import silhouette_score
from skimage import data
from skimage.filters import try_all_threshold
from skimage.filters import threshold_minimum
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [ ]:
import sys

sys.path.append('../src/data/')

from make_dataset import exportData, computeMNDWI, readData, computeNDWI
from etl import computeNDWI

Use this website to generate new JSON structures: https://geojson.io/#new&map=10.57/39.6198/-121.4357

In [ ]:
# might be useful later
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -121.57931418110947,
              39.73107014130147
            ],
            [
              -121.57931418110947,
              39.524185236048766
            ],
            [
              -121.28277910072802,
              39.524185236048766
            ],
            [
              -121.28277910072802,
              39.73107014130147
            ],
            [
              -121.57931418110947,
              39.73107014130147
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [ ]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

start_date = '2013-03-18'
end_date = '2022-10-24'

In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

Used some code from here: https://gis.stackexchange.com/questions/332259/calculate-ndwi-of-each-image-in-a-imagecollection

In [ ]:
# import Landsat 8 Level 2, Collection 2, Tier 1 surface reflectance
images_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start_date, end_date).filterBounds(aoi)
# filter cloudy images, threshold of 50%
images_sr = images_sr.filter(ee.Filter.lte('CLOUD_COVER', 0.80))

In [ ]:
# sentinel 
images_sentinel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate(start_date, end_date).filterBounds(aoi)
images_setinel = images_sentinel.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 0.20))

In [ ]:
# Landsat 7 Level 2, Collection 2, Tier 1 surface reflectance
images_landsat7 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start_date, end_date).filterBounds(aoi)
images_landsat7 = images_landsat7.filter(ee.Filter.lte('CLOUD_COVER', 0.80))

In [ ]:
# MOD44W.006 Terra Land Water Mask Derived From MODIS and SRTM Yearly Global 250m
modis_water_mask = ee.ImageCollection("MODIS/006/MOD44W").filterDate(start_date, end_date).filterBounds(aoi)

In [ ]:
# JRC Monthly Water History, v1.4
images_validation = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory").filterDate(start_date, end_date).filterBounds(aoi)

### Export Image Collection Section

##### Export landsat 7 image data

In [ ]:
landsat7 = images_landsat7.map(computeNDWI).select("NDWI")
collection_list = landsat7.toList(landsat7.size())
collection_size = collection_list.size().getInfo()
# # has a bug, has many dates that shouldnt be there. However, I will fix later
dates = geemap.image_dates(landsat7, date_format='YYYY-MM-dd').getInfo()

In [ ]:
# exportData(
#     aoi,
#     dates, 
#     collection_size, 
#     collection_list, 
#     "../data/LANDSAT7_NDWI/LANDSAT7_NDWI_{}.tif"
# )

##### Export modis water mask image data

In [ ]:
modis = modis_water_mask.select("water_mask")
collection_list = modis.toList(modis.size())
collection_size = collection_list.size().getInfo()
# # has a bug, has many dates that shouldnt be there. However, I will fix later
dates = geemap.image_dates(modis, date_format='YYYY-MM-dd').getInfo()

In [ ]:
# exportData(
#     aoi,
#     dates, 
#     collection_size, 
#     collection_list, 
#     "../data/WATER_MASK_DERIVED_MODIS/MODIS_WATER_MASK_{}.tif"
# )

##### Export setinel image data

In [ ]:
setinel = images_setinel.map(computeNDWI).select("NDWI")
collection_list = setinel.toList(setinel.size())
collection_size = collection_list.size().getInfo()
# # has a bug, has many dates that shouldnt be there. However, I will fix later
dates = geemap.image_dates(setinel, date_format='YYYY-MM-dd').getInfo()

In [ ]:
# exportData(
#     aoi,
#     dates, 
#     collection_size, 
#     collection_list, 
#     "../data/SENTINEL2_NDWI/SENTINEL2_NDWI_{}.tif"
# )

#### Band 3

In [ ]:
# SR_B3 band
sr_b3 = images_sr.select('SR_B3')
collection_list = sr_b3.toList(sr_b3.size())
collection_size = collection_list.size().getInfo()
# has a bug, has many dates that shouldnt be there. However, I will fix later
dates = geemap.image_dates(sr_b3, date_format='YYYY-MM-dd').getInfo()

In [ ]:
# exportData(
#     aoi,
#     dates, 
#     collection_size, 
#     collection_list, 
#     "../data/images/LANDSAT8_SR_B3/LANDSAT8_SR_B3_{}.tif"
# )

#### Band 5

In [ ]:
# SR_B5 band
sr_b5 = images_sr.select('SR_B5')
collection_list = sr_b5.toList(sr_b5.size())
collection_size = collection_list.size().getInfo()
# has a bug, has many dates that shouldnt be there. However, I will fix later
dates = geemap.image_dates(sr_b5, date_format='YYYY-MM-dd').getInfo()

In [ ]:
# exportData(
#     aoi,
#     dates, 
#     collection_size, 
#     collection_list, 
#     "../data/images/LANDSAT8_SR_B5/LANDSAT8_SR_B5_{}.tif"
# )

#### Band 6

In [ ]:
# SR_B6 band
sr_b6 = images_sr.select('SR_B6')
collection_list = sr_b6.toList(sr_b6.size())
collection_size = collection_list.size().getInfo()
# has a bug, has many dates that shouldnt be there. However, I will fix later
dates = geemap.image_dates(sr_b6, date_format='YYYY-MM-dd').getInfo()

In [ ]:
# exportData(
#     aoi,
#     dates, 
#     collection_size, 
#     collection_list, 
#     "../data/LANDSAT8_SR_B6/LANDSAT8_SR_B6_{}.tif"
# )

#### Band 7

In [ ]:
# SR_B7 band
sr_b7 = images_sr.select('SR_B6')
collection_list = sr_b7.toList(sr_b7.size())
collection_size = collection_list.size().getInfo()
# has a bug, has many dates that shouldnt be there. However, I will fix later
dates = geemap.image_dates(sr_b7, date_format='YYYY-MM-dd').getInfo()

In [ ]:
# exportData(
#     aoi,
#     dates, 
#     collection_size, 
#     collection_list, 
#     "../data/LANDSAT8_SR_B7/LANDSAT8_SR_B7_{}.tif"
# )

In [ ]:
# ndwi
ndwi_images = images_sr.map(computeNDWI).select("NDWI")
collection_list = ndwi_images.toList(ndwi_images.size())
collection_size = collection_list.size().getInfo()
# has a bug, has many dates that shouldnt be there. However, I will fix later
dates = geemap.image_dates(ndwi_images, date_format='YYYY-MM-dd').getInfo()

In [ ]:
# exportData(
#     aoi,
#     dates, 
#     collection_size, 
#     collection_list, 
#     "../data/LANDSAT8_NDWI_SR_B7/LANDSAT8_NDWI_B7_{}.tif"
# )

In [ ]:
# mndwi band
mndwi_images = images_sr.map(computeMNDWI).select("MNDWI")
collection_list = mndwi_images.toList(mndwi_images.size())
collection_size = collection_list.size().getInfo()
# has a bug, has many dates that shouldnt be there. However, I will fix later
dates = geemap.image_dates(mndwi_images, date_format='YYYY-MM-dd').getInfo()

In [ ]:
# exportData(
#     aoi,
#     dates, 
#     collection_size, 
#     collection_list, 
#     "../data/LANDSAT8_MNDWI/LANDSAT8_MNDWI_{}.tif"
# )

In [ ]:
# water band from JRC Monthly Water History
img_validation = images_validation.select("water")
collection_list = img_validation.toList(img_validation.size())
collection_size = collection_list.size().getInfo()
# has a bug, has many dates that shouldnt be there. However, I will fix later
dates = geemap.image_dates(img_validation, date_format='YYYY-MM-dd').getInfo()

In [ ]:
# exportData(
#     aoi,
#     dates, 
#     collection_size, 
#     collection_list, 
#     "../data/JRC_Monthly_Water_History/JRC_Monthly_{}.tif"
# )

### Read Data Section
Make a list of all gathered images in the form of (date, image) tuples.

In [ ]:
modis = readData("../data/WATER_MASK_DERIVED_MODIS/")
print(modis[0][0])
print(modis[0][1].shape)

In [ ]:
mndwi_img_lst = readData("../data/LANDSAT8_MNDWI/")
print(mndwi_img_lst[0][0])
print(mndwi_img_lst[0][1].shape)

In [ ]:
# list for sentinel data
ndwi_sentinel = readData("../data/SENTINEL2_NDWI/")
print(ndwi_sentinel[0][0])
print(ndwi_sentinel[0][1].shape)

In [ ]:
b3_img_lst = readData("../data/LANDSAT8_SR_B3/")
print(b3_img_lst[0][0])
print(b3_img_lst[0][1].shape)

In [ ]:
b5_img_lst = readData("../data/LANDSAT8_SR_B5/")
print(b5_img_lst[0][0])
print(b5_img_lst[0][1].shape)

In [ ]:
b6_img_lst = readData("../data/LANDSAT8_SR_B6/")
print(b6_img_lst[0][0])
print(b6_img_lst[0][1].shape)

In [ ]:
b7_img_lst = readData("../data/LANDSAT8_SR_B7/")
print(b7_img_lst[0][0])
print(b7_img_lst[0][1].shape)

In [ ]:
# list of landsat8 provide path to images
landsat8_ndwi_b5 = readData("../data/LANDSAT8_NDWI/")
print(landsat8_ndwi_b5[0][0])
print(landsat8_ndwi_b5[0][1].shape)

In [ ]:
landsat8_ndwi_b6 = readData("../data/LANDSAT8_NDWI_SR_B6/")
print(landsat8_ndwi_b6[0][0])
print(landsat8_ndwi_b6[0][1].shape)

In [ ]:
landsat8_ndwi_b7 = readData("../data/LANDSAT8_NDWI_SR_B7/")
print(landsat8_ndwi_b7[0][0])
print(landsat8_ndwi_b7[0][1].shape)

In [ ]:
jrc_monthly_water_history = readData("../data/JRC_Monthly_Water_History/")
print(jrc_monthly_water_history[0][0])
print(jrc_monthly_water_history[0][1].shape)

### Data Viz Section

#### NDWI image using sentinel data

In [ ]:
print(ndwi_sentinel[0][0])
plt.imshow(ndwi_sentinel[1][1])
plt.show()

plt.hist(
    ndwi_sentinel[1][1].flatten(), 
    bins=100
)
plt.ylabel("count")
plt.xlabel("intensity")
plt.title("Distribution of Sentinel pixel values: {}".format(ndwi_sentinel[0][0]))
plt.show()

In [ ]:
print(b3_img_lst[0][0])
plt.imshow(b3_img_lst[0][1])
plt.show()

plt.hist(
    b3_img_lst[0][1].flatten(), 
    bins=100
)
plt.ylabel("count")
plt.xlabel("intensity")
plt.title("Distribution of B3 pixel values: {}".format(b3_img_lst[0][0]))
plt.show()

In [ ]:
plt.hist(
    b3_img_lst[2][1].flatten(), 
    bins=80,
    alpha=0.4,
    label="Band B3",
    color="cornflowerblue"
)
plt.hist(
    b5_img_lst[2][1].flatten(), 
    bins=80,
    alpha=0.4,
    label="Band B5",
    color="mediumblue"
)
plt.hist(
    b6_img_lst[2][1].flatten(), 
    bins=80,
    alpha=0.4,
    label="Band B6",
    color="blueviolet"
)
plt.hist(
    b7_img_lst[2][1].flatten(), 
    bins=80,
    alpha=0.4,
    label="Band B7",
    color="blue"
)
plt.legend()
plt.ylabel("count")
plt.title("Distribution of SR_B3-SR_B7 pixel values: {}".format(b5_img_lst[2][0]))
plt.show()

fig, axes = plt.subplots(ncols=2, figsize=(10, 3))
ax = axes.ravel()

ax[0].imshow(b5_img_lst[1][1])
ax[0].set_title('Band 5: {}'.format(b5_img_lst[1][0]))

ax[1].imshow(b3_img_lst[1][1])
ax[1].set_title('Band 3: {}'.format(b3_img_lst[1][0]))


for a in ax:
    a.axis('off')

plt.show()

In [ ]:
plt.hist(
    b3_img_lst[1][1].flatten(), 
    bins=80,
    alpha=0.5,
    label="Band B3"
)
plt.hist(
    b6_img_lst[1][1].flatten(), 
    bins=80,
    alpha=0.5,
    label="Band B6"
)
plt.legend()
plt.ylabel("count")
plt.title("Distribution of B3 and B6 pixel values: {}".format(b6_img_lst[1][0]))
plt.show()

fig, axes = plt.subplots(ncols=2, figsize=(10, 3))
ax = axes.ravel()

ax[0].imshow(b6_img_lst[1][1])
ax[0].set_title('Band 6: {}'.format(b6_img_lst[1][0]))

ax[1].imshow(b3_img_lst[1][1])
ax[1].set_title('Band 3: {}'.format(b3_img_lst[1][0]))


for a in ax:
    a.axis('off')

plt.show()
plt.savefig(fname="savedfigs")

In [ ]:
plt.hist(
    b3_img_lst[1][1].flatten(), 
    bins=80,
    alpha=0.5,
    label="Band B3"
)
plt.hist(
    b7_img_lst[1][1].flatten(), 
    bins=80,
    alpha=0.5,
    label="Band B7"
)
plt.legend()
plt.ylabel("count")
plt.title("Distribution of B3 and B7 pixel values: {}".format(b7_img_lst[1][0]))
plt.show()

fig, axes = plt.subplots(ncols=2, figsize=(10, 3))
ax = axes.ravel()

ax[0].imshow(b7_img_lst[1][1])
ax[0].set_title('Band 7: {}'.format(b7_img_lst[1][0]))

ax[1].imshow(b3_img_lst[1][1])
ax[1].set_title('Band 3: {}'.format(b3_img_lst[1][0]))


for a in ax:
    a.axis('off')

plt.show()

In [ ]:
plt.hist(
    b3_img_lst[-1][1].flatten(), 
    bins=80,
    alpha=0.5,
    label="Band B3"
)
plt.hist(
    b5_img_lst[-1][1].flatten(), 
    bins=80,
    alpha=0.5,
    label="Band B5"
)

plt.legend()
plt.ylabel("count")
plt.title("Distribution of B3 and B5 pixel values: {}".format(b5_img_lst[-1][0]))
plt.show()

fig, axes = plt.subplots(ncols=2, figsize=(10, 3))
ax = axes.ravel()

ax[0].imshow(b5_img_lst[-1][1])
ax[0].set_title('Band 5: {}'.format(b5_img_lst[-1][0]))

ax[1].imshow(b3_img_lst[-1][1])
ax[1].set_title('Band 3: {}'.format(b3_img_lst[-1][0]))


for a in ax:
    a.axis('off')

plt.show()

In [ ]:
b3_mean_per_pixel = []
b5_mean_per_pixel = []
for i in range(1, len(b3_img_lst)):
    b3_mean_per_pixel.append(b3_img_lst[i][1])
    b5_mean_per_pixel.append(b5_img_lst[i][1])

b3_mean_per_pixel  = np.mean(
        np.dstack(b3_mean_per_pixel), 
        -1
    )
b5_mean_per_pixel  = np.mean(
        np.dstack(b5_mean_per_pixel), 
        -1
    )

plt.hist(
    b3_mean_per_pixel.flatten(), 
    bins=80,
    alpha=0.5,
    label="Band B3"
)
plt.hist(
    b5_mean_per_pixel.flatten(), 
    bins=80,
    alpha=0.5,
    label="Band B5"
)
plt.title("Distribution of B3 and B5 values")
plt.legend()
plt.ylabel("count")
plt.show()


fig, axes = plt.subplots(ncols=2, figsize=(10, 3))
ax = axes.ravel()

ax[0].imshow(b5_mean_per_pixel)
ax[0].set_title('Mean Band 5')

ax[1].imshow(b3_mean_per_pixel)
ax[1].set_title('Mean Band 3')


for a in ax:
    a.axis('off')

plt.show()

In [ ]:
plt.hist(
    jrc_monthly_water_history[1][1].flatten(), 
    bins=80,
    alpha=0.5,
    label="Validation Data"
)
# plt.hist(
#     landsat8_ndwi_b5[1][1].flatten(), 
#     bins=80,
#     alpha=0.5,
#     label="Band B5"
# )
plt.legend()
plt.ylabel("count")
plt.title("Distribution of pixel values for validation data: {}".format(jrc_monthly_water_history[1][0]))
plt.show()

fig, axes = plt.subplots(ncols=2, figsize=(10, 3))
ax = axes.ravel()

ax[0].imshow(landsat8_ndwi_b5[1][1])
ax[0].set_title('Band 5: {}'.format(landsat8_ndwi_b5[1][0]))

ax[1].imshow(jrc_monthly_water_history[1][1])
ax[1].set_title('Validation Data: {}'.format(jrc_monthly_water_history[1][0]))


for a in ax:
    a.axis('off')

plt.show()

In [ ]:
def visualize_images(img_lst, est_area_lst, binary):
    
    
    for i in range(len(img_lst)):

        fig, axes = plt.subplots(ncols=2, figsize=(8, 3))
        ax = axes.ravel()

        ax[0].imshow(img_lst[i][1], cmap=plt.cm.gray)
        ax[0].set_title('Orginal: {}'.format(d))

        ax[1].imshow(binary[i], cmap=plt.cm.gray)
        ax[1].set_title(
            'Result: {}\n Estimated Surface Area: {} meters squared'.format(img_lst[i][0], est_area_lst[i][1])
        )

        for a in ax:
            a.axis('off')

        plt.show()
    

In [ ]:
def estimate_surface_area(img_lst):
    
    out_list = []
    binary_out_lst = []

    for d, i in img_lst:
        thresh = threshold_minimum(i)
        binary = i > thresh
        binary_out_lst.append(binary)

        estimated_surface_area = (binary.flatten().sum() * (30 ** 2))
        out_list += [(d, estimated_surface_area)]
    
    return out_list, binary_out_lst
    
    

In [ ]:
landsat8_ndwi_b5_est_area, binary_b5 = estimate_surface_area(landsat8_ndwi_b5)
# displays viz of images and estimated surface area
visualize_images(
    landsat8_ndwi_b5,
    landsat8_ndwi_b5_est_area,
    binary_b5
)

In [ ]:
landsat8_ndwi_b6_est_area, binary_b6 = estimate_surface_area(landsat8_ndwi_b6)
# displays viz of images and estimated surface area
visualize_images(
    landsat8_ndwi_b6,
    landsat8_ndwi_b6_est_area,
    binary_b6
)

In [ ]:
landsat8_ndwi_b7_est_area, binary_b7 = estimate_surface_area(landsat8_ndwi_b7)
# displays viz of images and estimated surface area
visualize_images(
    landsat8_ndwi_b7,
    landsat8_ndwi_b7_est_area,
    binary_b7
)

In [ ]:
# this will resize the validation image data to match the same dimensions of our landsat data
resized_jrc = []

for d, i in jrc_monthly_water_history:
    resized_jrc.append((d, resize(i, landsat8_ndwi_b5[0][1].shape)))

In [ ]:
print(resized_jrc[0][1].shape)
plt.imshow(resized_jrc[0][1])

In [ ]:
plt.hist(
    resized_jrc[0][1]
)

In [ ]:
plt.imshow(resized_jrc[0][1] > 0.007)

In [ ]:
computed_area_valid, binary_valid = estimate_surface_area(resized_jrc)

In [ ]:
for i in range(len(binary_b5)):
    
    fig, axes = plt.subplots(ncols=2, figsize=(10, 3))
    ax = axes.ravel()

    ax[0].imshow(binary_b5[i])
    ax[0].set_title('NDWI using Band 3/5: {}'.format(landsat8_ndwi_b5[i][0]))

    ax[1].imshow(resized_jrc[i+2][1])
    ax[1].set_title('Validation Data: {}'.format(resized_jrc[i+2][0]))


    for a in ax:
        a.axis('off')

    plt.show()

In [ ]:
look into convert to decimal dates

In [ ]:
df1 = pd.DataFrame(landsat8_ndwi_b5_est_area, columns=['date', 'est_surface_area'])
df2 = pd.DataFrame(validation_data_area, columns=['date', 'est_surface_area'])

df1 = df1[df1["est_surface_area"] < 3e8]
df2 = df2[df2["est_surface_area"] > 0]


ax = df1.plot(x='date', y='est_surface_area', label='landsat8 NDWI')
df2.plot(x='date', y='est_surface_area', label='validation', ax=ax)

plt.show()

In [ ]:
for i in range(len(resized_jrc)):

        fig, axes = plt.subplots(ncols=2, figsize=(8, 3))
        ax = axes.ravel()

        ax[0].imshow(resized_jrc[i][1], cmap=plt.cm.gray)
        ax[0].set_title('Orginal: {}'.format(resized_jrc[i][0]))

        ax[1].imshow(resized_jrc[i][1] > 0.007, cmap=plt.cm.gray)
#         ax[1].set_title(
#             'Result: {}\n Estimated Surface Area: {} meters squared'.format(img_lst[i][0], est_area_lst[i][1])
#         )

        for a in ax:
            a.axis('off')

        plt.show()

In [ ]:
sorted(validation_data_area, key=lambda x: x[0])
plt.figure(figsize=(20, 5))
plt.ylabel('meters squared')
plt.xlabel('date')
plt.xticks(rotation=90)
d = [x[0] for x in validation_data_area if x[1] > 0]
g = [x[1] for x in validation_data_area if x[1] > 0]
plt.scatter(d, g, color="green")
plt.title('Validation Surface Area of Lake Oroville Over Time')
plt.show()

In [ ]:
sorted(landsat8_ndwi_b5_est_area, key=lambda x: x[0])
sorted(landsat8_ndwi_b6_est_area, key=lambda x: x[0])
sorted(landsat8_ndwi_b7_est_area, key=lambda x: x[0])
plt.figure(figsize=(20, 5))
plt.ylabel('meters squared')
plt.xlabel('date')
plt.xticks(rotation=90)
d = [x[0] for x in landsat8_ndwi_b5_est_area if x[1] < 2e8]
g = [x[1] for x in landsat8_ndwi_b5_est_area if x[1] < 2e8]
plt.scatter(d, g, color="red")
d = [x[0] for x in landsat8_ndwi_b6_est_area if x[1] < 2e8]
g = [x[1] for x in landsat8_ndwi_b6_est_area if x[1] < 2e8]
plt.scatter(d, g, color="blue")
d = [x[0] for x in landsat8_ndwi_b7_est_area if x[1] < 2e8]
g = [x[1] for x in landsat8_ndwi_b7_est_area if x[1] < 2e8]
plt.scatter(d, g, color="green")
plt.title('Estimated Surface Area of Lake Oroville Over Time')
plt.show()

In [ ]:
import pandas as pd

fr = pd.DataFrame(est_surface_areas, columns=['date', 'est_surface_area'])
fr['date'] = pd.to_datetime(fr['date'])
fr['year'] = fr['date'].dt.year
gb = fr.groupby('year').median()

plt.figure(figsize=(10, 10))
plt.ylabel('meters squared')
plt.xlabel('year')
plt.title('Median Estimated Water Surface Area of Lake Oroville by Year')
plt.bar(gb.index, gb.est_surface_area)
plt.show()

### Using Folium to display an image from an ImageCollection

In [ ]:
# import a collection and convert to an image
image = ee.Image(ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi))

In [ ]:
# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwi_viz = {'min': 0.4, 'max': 1, 'palette': ['blue', 'red']
}

In [ ]:
# Mask the non-watery parts of the image, where NDWI < 0.1.
ndwi_masked = ndwi.updateMask(ndwi.gte(0))

# Define a map centered on Lake Oroville.
map_ndwi_masked = folium.Map(location=[39.58, -121.47], zoom_start=11)

# Add the image layer to the map and display it.
map_ndwi_masked.add_ee_layer(image, {'min': .4, 'max': 1}, 'land mask')
map_ndwi_masked.add_ee_layer(ndwi_masked, ndwi_viz, 'NDWI masked')
display(map_ndwi_masked)

In [ ]:
image_rgb = image.visualize(**{'bands': ['B5', 'B4', 'B6'], 'max': 0.5})
ndwi_rgb = ndwi_masked.visualize(**{
    'min': 0.5,
    'max': 1,
    'palette': ['00FFFF', '0000FF']
})

In [ ]:
# Mosaic the visualization layers and display (or export).
mosaic = ee.ImageCollection([image_rgb, ndwi_rgb]).mosaic()

# Define a map centered on Lake Oroville.
map_mosaic = folium.Map(location=[39.58, -121.47], zoom_start=11)

# Add the image layer to the map and display it.
map_mosaic.add_ee_layer(mosaic, None, 'mosaic')
display(map_mosaic)